In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from Bio import SeqIO
import json
import requests

In [5]:
smash = pd.read_csv("../misc_data/antismash_db_results.csv", sep="\t")
smash[smash["NCBI accession"] == "NC_010612.1"] 
counts = smash["NCBI accession"].value_counts()
above_10_index = counts[counts > 5].index
smash = smash[smash["NCBI accession"].isin(above_10_index)]
smash["NCBI accession"] = smash["NCBI accession"].apply(lambda row: row.split(".")[0])
smash_dict = dict()
for accession, df in smash.groupby("NCBI accession"):
    smash_dict[accession] = df[["From", "To"]].to_numpy()

In [11]:
len(counts.index)

384

In [13]:
f = open("antismash_accessions_greater_than_5_bgcs.txt", "w")
for i in smash_dict.keys():
    f.write(i + "\n")
f.close()

In [ ]:
smash_dict

In [5]:
import random


In [42]:
num_splits = 5
k_split = 1
top_k = 100

def get_splits(k_split, num_splits=5, top_k = 100):

    smash = pd.read_csv("../misc_data/antismash_db_results.csv", sep="\t")
    top_k_accessions = smash["NCBI accession"].value_counts()[:top_k].index
    # above_10_index = counts[counts > 5].index
    smash = smash[smash["NCBI accession"].isin(top_k_accessions)]
    smash["NCBI accession"] = smash["NCBI accession"].apply(lambda row: row.split(".")[0])
    smash_dict = dict()
    for accession, df in smash.groupby("NCBI accession"):
        smash_dict[accession] = df[["From", "To"]].to_numpy()
    smash_nums = sorted([(k, len(value)) for k, value in smash_dict.items()], key = lambda tup: tup[1], reverse=True)
    random.seed(42)
    random.shuffle(smash_nums)
    splits = [[] for i in range(num_splits)]
    for ix, tup in enumerate(smash_nums):
        split = ix % num_splits
        splits[split].append(tup[0])

    test = splits.pop(k_split)
    train = [i for s in splits for i in s]
    
    test = subprocess.run(["../scripts/genome_accession_to_patric_genome_ids.sh"] + test, capture_output=True).stdout.decode("utf-8").split("\n")
    test = [i for i in test if i != ""]
    
    train = subprocess.run(["../scripts/genome_accession_to_patric_genome_ids.sh"] + train, capture_output=True).stdout.decode("utf-8").split("\n")
    train = [i for i in train if i != ""]
    
    return train, test

In [ ]:
get_splits(0)

In [34]:
splits = [[] for i in range(num_splits)]
for ix, tup in enumerate(smash_nums):
    split = ix % num_splits
    splits[split].append(tup[0])
    
test = splits.pop(k_split)
train = [i for s in splits for i in s]

In [26]:
len(smash_dict.keys())

100

In [ ]:
smash_nums

In [7]:
import subprocess

In [91]:
subprocess.run(f"rg $(rg {'|'.join(top)} ../misc_data/prokaryotes.txt | cut -f 19) ../notebooks/genome_metadata | cut -f 1 ".split(), capture_output=True).stderr

b'19): No such file or directory (os error 2)\n'

In [41]:
subprocess.run(["../scripts/genome_accession_to_patric_genome_ids.sh"] + top, capture_output=True).stdout.decode("utf-8").split("\n")

['439334.465',
 '189918.11',
 '2094119.3',
 '53376.9',
 '1781.42',
 '120959.23',
 '1767.9',
 '1547487.3',
 '']

In [ ]:
subprocess.run("rg 'NZ_CP011883|NZ_MPNT01000022|NZ_SDLM01000041|NZ_BFCH01000017|NZ_QAYH01000003|NZ_BFAB01000032|NZ_CP023152|NZ_BFCH01000002|NZ_AP022581|NZ_SDLM01000005' ../misc_data/prokaryotes.txt | grep -v Mycobacterium".split(),capture_output=True).stdout #| cut -f 19 | tr -d '\n' | sed 's/.[0-9]G/|G/g'".split(), capture_output=True, shell=True).stderr.decode("ascii")

In [94]:
f"rg $(rg {'|'.join(top)} ../misc_data/prokaryotes.txt | cut -f 19) ../notebooks/genome_metadata | cut -f 1 "

'rg $(rg NZ_CP011883|NZ_MPNT01000022|NZ_SDLM01000041|NZ_BFCH01000017|NZ_QAYH01000003|NZ_BFAB01000032|NZ_CP023152|NZ_BFCH01000002|NZ_AP022581|NZ_SDLM01000005 ../misc_data/prokaryotes.txt | cut -f 19) ../notebooks/genome_metadata | cut -f 1 '

In [12]:
def genome_ids_from_accessions(accessions=[], limit=250000000, stream=False):
    
    selectors = ["ne(feature_type)","eq(annotation,PATRIC)",f"in(accession,({','.join(accessions)}))"]
    genomes = f"and({','.join(selectors)})"    
    limit = f"limit({limit})"
    select = "select(genome_id,genome_name,accession,annotation,feature_type,patric_id,refseq_locus_tag,alt_locus_tag,uniprotkb_accession,start,end,strand,na_length,gene,product,figfam_id,plfam_id,pgfam_id,go,ec,pathway,aa_sequence_md5)&sort(+genome_id,+sequence_id,+start)"
    base = "https://www.patricbrc.org/api/genome_feature/"
    query = "&".join([genomes, limit, select])
    headers = {"accept":"text/tsv", "content-type": "application/rqlquery+x-www-form-urlencoded"}

    genome_ids = []
    
    r = requests.post(url=base, data=query, headers=headers, stream=stream)
    
    if r.encoding is None:
        r.encoding = "utf-8"
    for ix, line in enumerate(r.iter_lines(decode_unicode=True)):
        line = line.replace('"', '')
        if ix > 0: 
            genome_ids.append(line.split()[0])
    return list(set(genome_ids))

In [60]:
def chunker(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i+chunk_size]

In [61]:
def tsv_from_genome_ids(outfile, genome_ids, limit=25000000, stream=False):
    with open(outfile, "w") as dest:
        pass
    for gids in chunker(genome_ids, 1):
        selectors = ["ne(feature_type)","eq(annotation,PATRIC)",f"in(genome_id,({','.join(gids)}))"]
    #     selectors = ["ne(feature_type)","eq(annotation,PATRIC)",f"in(accession,({','.join(accessions)}))"]
        genomes = f"and({','.join(selectors)})"    
        limit = f"limit({limit})"
        select = "select(genome_id,genome_name,accession,annotation,feature_type,patric_id,refseq_locus_tag,alt_locus_tag,uniprotkb_accession,start,end,strand,na_length,gene,product,figfam_id,plfam_id,pgfam_id,go,ec,pathway,aa_sequence_md5)&sort(+genome_id,+sequence_id,+start)"
        base = "https://www.patricbrc.org/api/genome_feature/"
        query = "&".join([genomes, limit, select])
        headers = {"accept":"text/tsv", "content-type": "application/rqlquery+x-www-form-urlencoded"}

        genome_ids = []

        r = requests.post(url=base, data=query, headers=headers, stream=stream)

        if r.encoding is None:
            r.encoding = "utf-8"
        with open(outfile, "a") as dest:
            dest.write(r.text)

In [60]:
anti_gids = []
with open("../misc_data/antismash_genome_from_accession_no_plasmid_unique.txt") as src:
    for line in src.readlines():
        anti_gids.append(line.strip())

In [75]:
tsv_from_genome_ids("anti_gids_no_pfams.tsv", anti_gids[:-1])

In [58]:
top_3_anti_gids = []
with open("antismash_top_3_accession_gids.txt") as src:
    for line in src.readlines():
        top_3_anti_gids.append(line.strip())
top_3_anti_gids

['216594.6', '1781.42', '1136880.4']

In [62]:
tsv_from_genome_ids("top_3_anti_gids_no_pfams.tsv", top_3_anti_gids)

In [11]:
print(len(smash_dict.keys()))

90


In [23]:
# with open("antismash_genome_ids.txt", "w") as dest:
with open("antismash_accession_ids.txt", "w") as dest:
#     for i in smash_genome_ids:
    for i in smash_dict.keys():
        dest.write(i + "\n")

In [3]:
records = [r for r in SeqIO.parse("/sfs/lustre/bahamut/scratch/jho5ze/bionets/BGCs/genera/Mycobacterium/jsonhfasta/2021-02-24.jsonhfasta", "fasta")]
seq_dict = [json.loads(r.description.split(None, 1)[-1]) for r in records]
seq_dict = {x["md5"]:sorted([i for i in x.setdefault("sequence_domain", [{"source":"none", "start":"0"}]) if i["source"] == "pfam"], key = lambda x: int(x["start"])) for x in seq_dict}

In [55]:
def gen_pfam_graph_input(source, destination):
    with open(source) as src:
        with open(destination, "w") as dest:
            for ix, line in enumerate(src.readlines()):
                vals = line.strip().split("\t")
                if ix == 0:
                    names = vals
                    name_ids = {name:ix for ix, name in enumerate(names)}
                try:
                    vals = {name:vals[ix] for ix, name in enumerate(names)}
                except:
                    continue
                md5 = vals["aa_sequence_md5"].replace('"', '')
                pgfam = vals["pgfam_id"]

                #This means that the pgfam is NAN and should be skipped
                if len(pgfam) < 2:
                    continue

                dest.write(line)

                try:
                    pfams = seq_dict[md5]
                except Exception as e:
                    continue
                if len(pfams) > 0:
                    for pfam in pfams:
                        pfam_line = line.split("\t")
                        pfam_line[name_ids["start"]] = str(int(vals['start']) + int(pfam['start']))
                        pfam_line[name_ids["end"]] = str(int(vals['start']) + int(pfam['end']))
                        pfam_line[name_ids["na_length"]] = str(int(pfam['end']) - int(pfam['start']))
                        pfam_line[name_ids["pgfam_id"]] = pfam["sequence_domain"]
                        pfam_line[name_ids["figfam_id"]] = pfam["sequence_domain"]
                        pfam_line[name_ids["plfam_id"]] = pfam["sequence_domain"]
                        pfam_line[name_ids["annotation"]] = "CUSTOM"
                        dest.write(("\t".join(pfam_line)))


In [56]:
gen_pfam_graph_input("anti_gids_no_pfams.tsv", "anti_gids_with_pfams.tsv")

In [63]:
gen_pfam_graph_input("top_3_anti_gids_no_pfams.tsv", "top_3_anti_gids_with_pfams.tsv")